In [13]:
from credentials import *
import tweepy
print(tweepy.__version__)

3.10.0


In [2]:
## credentials contain:
customer_key        = "oJPfWgPQWoTmmcDV8wt7XQiJm"
customer_secret     = "4ievAUWvc87v6wgOh7IvqTVtvLnJ6esvvZjfkVTgy62g8MvvHL"
access_token        = "1480765674532192257-JCXhWc1t7QtEmLBKMtsQ9sqZFhPaqy"
access_token_secret = "kmBKSuixBf9FimkuAvlbQd372rf4criee246H36MUQdsy"

In [4]:
userID = "VastBlast"

In [5]:
auth = tweepy.OAuthHandler(customer_key, customer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

In [25]:
all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id
while True:
    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           max_id = oldest_id - 1,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 389
N of tweets downloaded till now 588
N of tweets downloaded till now 783
N of tweets downloaded till now 980
N of tweets downloaded till now 1178
N of tweets downloaded till now 1374
N of tweets downloaded till now 1567
N of tweets downloaded till now 1764
N of tweets downloaded till now 1962
N of tweets downloaded till now 2161
N of tweets downloaded till now 2358
N of tweets downloaded till now 2558
N of tweets downloaded till now 2757
N of tweets downloaded till now 2955
N of tweets downloaded till now 3155
N of tweets downloaded till now 3182


In [26]:
#transform the tweepy tweets into a 2D array that will populate the csv	
from pandas import DataFrame
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]
df = DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])
df.to_csv('%s_tweets.csv' % userID,index=False)
df.head(3)

,id,created_at,favorite_count,retweet_count,text
0,1481757976159531014,2022-01-13 22:40:05+00:00,8,0,"@Foolsihh Yep, but if you actually think about..."
1,1481751182972489728,2022-01-13 22:13:06+00:00,259,47,"People argue that Bitcoin ""comes out of thin a..."
2,1479564736752766977,2022-01-07 21:24:56+00:00,2047,421,You can only choose ONE 🐕\n\n#Shib\n$Floki\n$D...


In [27]:
import datetime as dt
import pandas as pd

data = pd.read_csv('WatcherGuru_tweets.csv')


In [34]:
data['created_at'] = pd.to_datetime(data['created_at'])
filtered_df = data.loc[(data['created_at'] >= '2022-01-06')]

id_list = filtered_df['id'].to_list()

In [22]:
# update these for whatever tweet you want to process replies to
name = 'WatcherGuru'
tweet_id = '1481554511466540000'

replies=[]
for tweet in tweepy.Cursor(api.search,q='to:'+name, result_type='recent', timeout=999999).items(1000):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            replies.append(tweet)

with open('replies_clean.csv', 'w') as f:
    csv_writer = csv.DictWriter(f, fieldnames=('user', 'text'))
    csv_writer.writeheader()
    for tweet in replies:
        row = {'user': tweet.user.screen_name, 'text': tweet.text.replace('\n', ' ')}
        csv_writer.writerow(row)



In [20]:
replies

[]

In [10]:
import csv

with open('replies_clean.csv', 'w') as f:
    csv_writer = csv.DictWriter(f, fieldnames=('user', 'text'))
    csv_writer.writeheader()
    for tweet in replies:
        row = {'user': tweet.user.screen_name, 'text': tweet.text.replace('\n', ' ')}
        csv_writer.writerow(row)